In [1]:
import json
import sys
import os
import csv
import glob


In [2]:
def load(filename):
    with open(filename, 'r') as infile:
        data = json.load(infile)
        return data

with open('../public/data/Child Care Desert binary variable.csv', 'r') as incsv:
    deserts = { r['ZCTA']: int(r['ccdesert']) for r in csv.DictReader(incsv) }

with open('../public/data/childCareCenters.tsv', 'r') as incsv:
    demographics = { r['zip']: r for r in csv.DictReader(incsv, delimiter="\t") }

with open('../public/data/zips_urbanicity.csv') as incsv:
    urbanicity = { r['ZCTA']: r for r in csv.DictReader(incsv) }



In [3]:
#
# Join zip csvs with geojson
#
topopath = '../public/data/zcta/'
zippath = '../public/data/zipdata/'
for filename in glob.glob(topopath + '*'):
    data = load(filename)
    stateId = None
    for geometry in data['features']:
        zipcode = geometry['properties']['ZCTA5CE10']
        if zipcode in deserts:
            geometry['properties']['ccd'] = deserts[zipcode]
        if zipcode in demographics:
            dem = demographics[zipcode]
            if (stateId == None):
                stateId = dem['state']
#             geometry['properties']['dem'] = {
#                 "b": dem['black'],
#                 "l": dem['latino'],
#                 "d": dem['density'],
#                 "e": dem['meanearnin']
#             }
        if zipcode in urbanicity:
            urb = urbanicity[zipcode]
            geometry['properties']['urb'] = urbanicity[zipcode]['ZIP_type']
        
    with open(zippath + "zip_" + stateId.zfill(2) + ".geo.json", 'w') as outfile:
        json.dump(data, outfile)